In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import furniture_bench
from furniture_bench.envs.furniture_sim_env import FurnitureSimEnv
import os
from pathlib import Path

import torch

import numpy as np
import matplotlib.pyplot as plt

from src.visualization.render_mp4 import (
    mp4_from_pickle_jupyter,
    unpickle_data,
    mp4_from_data_dict_jupyter,
    create_mp4_jupyter,
)
from src.common.geometry import action_to_6d_rotation
from src.data_processing.utils import resize, resize_crop

from tqdm import trange


from src.gym import get_env

Importing module 'gym_38' (/data/scratch/ankile/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /data/scratch/ankile/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.0.1+cu117
Device count 1
/data/scratch/ankile/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /data/scratch/ankile/.cache as PyTorch extensions root...
Emitting ninja build file /data/scratch/ankile/.cache/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...


In [3]:
env: FurnitureSimEnv = get_env(
    furniture="one_leg",
    num_envs=1,
    gpu_id=0,
    act_rot_repr="rot_6d",
    ctrl_mode="osc",
    randomness="low",
    resize_img=False,
)

In [4]:
tmp_dir = Path("tmp_rollout_dir")

In [5]:
scripted_dir = (
    Path(os.environ["DATA_DIR_RAW"])
    / "raw"
    / "sim"
    / "one_leg"
    / "scripted"
    / "low"
    / "success"
)

scripted_pickles = list(scripted_dir.glob("*.pkl*"))

len(scripted_pickles)

250

In [6]:
scripted_data = [unpickle_data(p) for p in scripted_pickles[:10]]

In [9]:
# Play them back in the simulator
for i, data in enumerate(scripted_data[:5]):
    # Reset the env
    env.reset()

    # Set the initial state
    env.reset_env_to(env_idx=0, state=data["observations"][0])

    # Play back the actions in the trajectory
    imgs1 = []
    imgs2 = []

    actions = np.array(data["actions"])
    action = action_to_6d_rotation(torch.tensor(actions).cuda())

    for i in trange(len(actions)):
        obs, reward, done, info = env.step(action[i : i + 1])
        imgs1.append(resize(obs["color_image1"]).squeeze().cpu().numpy())
        imgs2.append(resize_crop(obs["color_image2"]).squeeze().cpu().numpy())

    # See the scripted and the replay side-by-side
    ims2_s = np.array([o["color_image2"] for o in data["observations"]])[:-1]
    ims2_r = np.array(imgs2)

    print(ims2_s.shape, ims2_r.shape)

    ims_sbs = np.concatenate([ims2_s, ims2_r], axis=2)

    create_mp4_jupyter(ims_sbs, tmp_dir / f"scripted_replay{i}.mp4")

INFO: Found collision-free init pose
100%|██████████| 614/614 [00:28<00:00, 21.23it/s]


(614, 240, 320, 3) (614, 240, 320, 3)
File saved as tmp_rollout_dir/scripted_replay613.mp4


INFO: Found collision-free init pose
100%|██████████| 656/656 [00:28<00:00, 23.34it/s]


(656, 240, 320, 3) (656, 240, 320, 3)
File saved as tmp_rollout_dir/scripted_replay655.mp4


INFO: Found collision-free init pose
100%|██████████| 654/654 [00:27<00:00, 24.01it/s]


(654, 240, 320, 3) (654, 240, 320, 3)
File saved as tmp_rollout_dir/scripted_replay653.mp4


INFO: Found collision-free init pose
100%|██████████| 605/605 [00:27<00:00, 21.93it/s]


(605, 240, 320, 3) (605, 240, 320, 3)
File saved as tmp_rollout_dir/scripted_replay604.mp4


INFO: Found collision-free init pose
100%|██████████| 609/609 [00:27<00:00, 22.53it/s]


(609, 240, 320, 3) (609, 240, 320, 3)
File saved as tmp_rollout_dir/scripted_replay608.mp4


In [ ]:
del env

In [ ]:
env: FurnitureSimEnv = get_env(
    furniture="one_leg",
    num_envs=1,
    gpu_id=0,
    act_rot_repr="quat",
    ctrl_mode="osc",
    randomness="low",
    resize_img=False,
)

INFO: Making new env: FurnitureSim-v0 ({'furniture': 'one_leg', 'num_envs': 1, 'resize_img': False, 'concat_robot_state': True, 'headless': True, 'obs_keys': ['robot_state/ee_pos', 'robot_state/ee_quat', 'robot_state/ee_pos_vel', 'robot_state/ee_ori_vel', 'robot_state/gripper_width', 'color_image1', 'color_image2', 'parts_poses'], 'compute_device_id': 0, 'graphics_device_id': 0, 'init_assembled': False, 'np_step_out': False, 'channel_first': False, 'randomness': 'low', 'high_random_idx': -1, 'save_camera_input': False, 'record': False, 'max_env_steps': 3000, 'act_rot_repr': 'quat', 'ctrl_mode': 'osc', 'verbose': False})


In [ ]:
# Play them back in the simulator
for i, data in enumerate(scripted_data[:5]):
    # Reset the env
    env.reset()

    # Set the initial state
    env.reset_env_to(env_idx=0, state=data["observations"][0])

    # Play back the actions in the trajectory
    imgs1 = []
    imgs2 = []

    actions = np.array(data["actions"])
    # action = action_to_6d_rotation(torch.tensor(actions).cuda())
    action = torch.tensor(actions).cuda()

    for i in trange(len(actions)):
        obs, reward, done, info = env.step(action[i : i + 1])
        imgs1.append(resize(obs["color_image1"]).squeeze().cpu().numpy())
        imgs2.append(resize_crop(obs["color_image2"]).squeeze().cpu().numpy())

    # See the scripted and the replay side-by-side
    ims2_s = np.array([o["color_image2"] for o in data["observations"]])[:-1]
    ims2_r = np.array(imgs2)

    print(ims2_s.shape, ims2_r.shape)

    ims_sbs = np.concatenate([ims2_s, ims2_r], axis=2)

    create_mp4_jupyter(ims_sbs, tmp_dir / f"scripted_replay{i}.mp4")

INFO: Found collision-free init pose
100%|██████████| 614/614 [00:25<00:00, 23.98it/s]


(614, 240, 320, 3) (614, 240, 320, 3)
File saved as tmp_rollout_dir/scripted_replay613.mp4


INFO: Found collision-free init pose
100%|██████████| 656/656 [00:27<00:00, 23.59it/s]


(656, 240, 320, 3) (656, 240, 320, 3)
File saved as tmp_rollout_dir/scripted_replay655.mp4


INFO: Found collision-free init pose
100%|██████████| 654/654 [00:26<00:00, 24.33it/s]


(654, 240, 320, 3) (654, 240, 320, 3)
File saved as tmp_rollout_dir/scripted_replay653.mp4


INFO: Found collision-free init pose
100%|██████████| 605/605 [00:25<00:00, 23.28it/s]


(605, 240, 320, 3) (605, 240, 320, 3)
File saved as tmp_rollout_dir/scripted_replay604.mp4


INFO: Found collision-free init pose
100%|██████████| 609/609 [00:25<00:00, 23.48it/s]


(609, 240, 320, 3) (609, 240, 320, 3)
File saved as tmp_rollout_dir/scripted_replay608.mp4
